In [1]:
%pip install plotly
import pandas as pd
from pandas import read_csv
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np
# for mpl animation
import matplotlib.animation as animation
from matplotlib import rc
from matplotlib import collections  as mc
import plotly.graph_objects as go
from scipy.stats import multivariate_normal
from copy import deepcopy
%matplotlib inline
rc('animation', html='html5')
games = read_csv('games.csv')
tracking_df = read_csv('tracking_week_1.csv')
play_df = read_csv('plays.csv')
players = read_csv('players.csv')
tackles = read_csv('tackles.csv')
tracking_data = tracking_df
plays = play_df
tracking_data['x'] = tracking_data.apply(lambda row : row['x'] if row['playDirection'] == 'right' else 120-row['x'], axis = 1)
tracking_data['y'] = tracking_data.apply(lambda row : row['y'] if row['playDirection'] == 'right' else 160/3-row['y'], axis = 1)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [92]:
import warnings
warnings.filterwarnings("ignore")

def get_board(gameId, playId, frameId):
    play = tracking_data[tracking_data['playId'] == playId][tracking_data['gameId'] == gameId][tracking_data['frameId'] == frameId]
    offenseTeam = plays[plays['playId'] == playId][plays['gameId'] == gameId]['possessionTeam'].values[0]
    defenseTeam = plays[plays['playId'] == playId][plays['gameId'] == gameId]['defensiveTeam'].values[0]
    offense = play[play['club'] == offenseTeam]
    defense = play[play['club'] == defenseTeam]

    carrierId, tacklerId = get_carrier_tackler(gameId, playId)
    offense_pos = get_player_position(gameId, playId, frameId, carrierId)
    defense_pos = get_player_position(gameId, playId, frameId, tacklerId)

    grid_offense = np.zeros((53, 120))
    for x, y in zip(offense['x'], offense['y']):
        if (y, x) != offense_pos and (y, x) != defense_pos:
            for offsetx in [-1, 0, 1]:
                for offsety in [-1, 0, 1]:
                    grid_offense[np.clip(int(y) + offsety, 0, 52), np.clip(int(x) + offsetx, 0, 119)] = 1

    grid_defense = np.zeros((53, 120))
    for x, y in zip(defense['x'], defense['y']):
        if (y, x) != offense_pos and (y, x) != defense_pos:
            for offsetx in [-1, 0, 1]:
                for offsety in [-1, 0, 1]:
                    grid_defense[np.clip(int(y) + offsety, 0, 52), np.clip(int(x) + offsetx, 0, 119)] = 1

    
    return np.array([offense['y'].values, offense['x'].values]).T, np.array([defense['y'].values, defense['x'].values]).T#grid_offense, grid_defense

def grid(values, pos, tol = 1.):
    for player_pos in values:
        if np.linalg.norm(player_pos - pos) < tol:
            return False
    return True
    
def get_carrier_tackler(gameId, playId):
    return plays[plays['playId'] == playId][plays['gameId'] == gameId]['ballCarrierId'].values[0], tackles[tackles['gameId'] == gameId][tackles['playId'] == playId]['nflId'].values[0]

def get_los(gameId, playId, ballCarrier):
    #playDirection = tracking_data[tracking_data['playId'] == playId][tracking_data['gameId'] == gameId]['playDirection'].values[0]
    #line_of_scrimmage = plays[plays['gameId'] == gameId][plays['playId'] == playId]['yardlineNumber'].values[0]
    #if playDirection == 'right':
    #    return line_of_scrimmage
    #return 100 - line_of_scrimmage
    
    return tracking_data[tracking_data['playId'] == playId][tracking_data['gameId'] == gameId][tracking_data['frameId'] == 1][tracking_data['nflId'] == ballCarrier]['x'].values[0]

    
def get_legal_moves(values, pos):
    legal_moves = []
    y = pos[0]
    x = pos[1]
    distances = np.linspace(1, 10, num = 3) / 6.
    angles = np.linspace(0, 2 * np.pi, num = 4)
    for distance in distances:
        for angle in angles:
            offsety = distance * math.sin(angle)
            offsetx = distance * math.cos(angle)
            newy = int(y + offsety)
            newx = int(x + offsetx)
            if 0 <= newy and newy < 53 and 0 <= newx and newx < 120:
                if grid(values, np.array([y + offsety, x + offsetx])):
                    legal_moves.append(np.array([y + offsety, x + offsetx]))
                    
    return legal_moves


def minimax(depth, grid_offense, grid_defense, offense_pos, defense_pos, max_or_min, alpha, beta, los, offense_cells, defense_cells, transpositions):
    if depth == 0 or np.linalg.norm(np.array(offense_pos) - np.array(defense_pos)) < 2:
        return offense_pos[1] - los, offense_cells, defense_cells
    best_score = -121.
    best_moves_offense = []
    best_moves_defense = []
    if str(grid_offense.round(decimals = 2)) + str(grid_defense.round(decimals = 2)) in transpositions:
        best_score, offense_moves, defense_moves, alpha, beta = transpositions[str(grid_offense.round(decimals = 2)) + str(grid_defense.round(decimals = 2))]
        return (best_score, offense_moves, defense_moves)
        
    if not max_or_min:#is minimizer, defense
        best_score = best_score * -1

        moves = get_legal_moves(grid_defense, np.array(defense_pos))
        if len(moves) == 0:
            return offense_pos[1] - los, offense_cells, defense_cells
        moves = sorted(moves, key = lambda x: np.linalg.norm(x - np.array(offense_pos)))
        for move in moves:
            new_defense_pos = move
            score, offensive_cells, defensive_cells = minimax(depth - 1, grid_offense, grid_defense, offense_pos, new_defense_pos, not max_or_min, alpha, beta, los, offense_cells, defense_cells + new_defense_pos.tolist(), transpositions)
            if score < best_score:
                best_score = score
                best_moves_offense = offensive_cells
                best_moves_defense = defensive_cells
            beta = min(beta, best_score)
            if beta <= alpha:
                transpositions[str(grid_offense.round(decimals = 2)) + str(grid_defense.round(decimals = 2))] = (best_score, best_moves_offense, best_moves_defense, alpha, beta)
                return best_score, best_moves_offense, best_moves_defense
                
    else:#is maximizer, offense
        moves = get_legal_moves(grid_offense, np.array(offense_pos))
        if len(moves) == 0:
            return offense_pos[1] - los, offense_cells, defense_cells
        
        #moves = sorted(moves, key = lambda x: -np.linalg.norm(x - np.array(defense_pos)))
        for move in moves:
            new_offense_pos = move
            score, offensive_cells, defensive_cells = minimax(depth - 1, grid_offense, grid_defense, new_offense_pos, defense_pos, not max_or_min, alpha, beta, los, offense_cells + new_offense_pos.tolist(), defense_cells, transpositions)
            if score > best_score:
                best_score = score
                best_moves_offense = offensive_cells
                best_moves_defense = defensive_cells
            alpha = max(alpha, best_score)

            if beta <= alpha:
                transpositions[str(grid_offense.round(decimals = 2)) + str(grid_defense.round(decimals = 2))] = (best_score, best_moves_offense, best_moves_defense, alpha, beta)
                return best_score, best_moves_offense, best_moves_defense
    return best_score, best_moves_offense, best_moves_defense

def get_player_position(gameId, playId, frameId, playerId):
    play = tracking_data[tracking_data['playId'] == playId][tracking_data['gameId'] == gameId][tracking_data['frameId'] == frameId]
    x = play[play['nflId'] == playerId]['x'].values[0]
    y = play[play['nflId'] == playerId]['y'].values[0]
    return y, x
gameId = 2022091100
playId = 1887
from time import time
from tqdm import tqdm
def get_evals(gameId, playId):
    num_frames = len(set(tracking_data[tracking_data['playId'] == playId][tracking_data['gameId'] == gameId]['frameId'].values))
    evals = []
    for i in tqdm(range(1, num_frames)):
        g_offense, g_defense = get_board(gameId, playId, i)
        #print(str(g_offense.round(decimals = 0)))
        import math
        carrierId, tacklerId = get_carrier_tackler(gameId, playId)
        offense_pos = get_player_position(gameId, playId, i, carrierId)
        defense_pos = get_player_position(gameId, playId, i, tacklerId)
        #start = time()
        yards, offense_cells, defense_cells = minimax(100, g_offense, g_defense, offense_pos, defense_pos, True, -math.inf, math.inf, get_los(gameId, playId, carrierId), [], [], {})
        evals.append(yards)
        #print('Yards gained ', i, ': ', yards, len(offense_cells) * .5 + len(defense_cells) * .5, ' Time: ', time() - start)
    return evals

def sign(x):
    if x < 0:
        return -1
    return 1
    
def get_keypoints(gameId, playId):
    evals = get_evals(gameId, playId)
    keyframes = []
    for i in range(1, len(evals) - 1):
        if sign(evals[i + 1] - evals[i]) != sign(evals[i] - evals[i - 1]):
            keyframes.append(i)
    return keyframes, evals

In [ ]:
def ColorDistance(hex1,hex2):
    '''d = {} distance between two colors(3)'''
    if hex1 == hex2:
        return 0
    rgb1 = hex_to_rgb_array(hex1)
    rgb2 = hex_to_rgb_array(hex2)
    rm = 0.5*(rgb1[0]+rgb2[0])
    d = abs(sum((2+rm,4,3-rm)*(rgb1-rgb2)**2))**0.5
    return d


def hex_to_rgb_array(hex_color):
    '''take in hex val and return rgb np array'''
    return np.array(tuple(int(hex_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)))

def ColorPairs(team1,team2):
    color_array_1 = colors[team1]
    color_array_2 = colors[team2]
    # If color distance is small enough then flip color order
    if ColorDistance(color_array_1[0],color_array_2[0])<500:
        return {team1:[color_array_1[0],color_array_1[1]],team2:[color_array_2[1],color_array_2[0]],'football':colors['football']}
    else:
        return {team1:[color_array_1[0],color_array_1[1]],team2:[color_array_2[0],color_array_2[1]],'football':colors['football']}
colors = {
    'ARI':["#97233F","#000000","#FFB612"], 
    'ATL':["#A71930","#000000","#A5ACAF"], 
    'BAL':["#241773","#000000"], 
    'BUF':["#00338D","#C60C30"], 
    'CAR':["#0085CA","#101820","#BFC0BF"], 
    'CHI':["#0B162A","#C83803"], 
    'CIN':["#FB4F14","#000000"], 
    'CLE':["#311D00","#FF3C00"], 
    'DAL':["#003594","#041E42","#869397"],
    'DEN':["#FB4F14","#002244"], 
    'DET':["#0076B6","#B0B7BC","#000000"], 
    'GB' :["#203731","#FFB612"], 
    'HOU':["#03202F","#A71930"], 
    'IND':["#002C5F","#A2AAAD"], 
    'JAX':["#101820","#D7A22A","#9F792C"], 
    'KC' :["#E31837","#FFB81C"], 
    'LA' :["#003594","#FFA300","#FF8200"], 
    'LAC':["#0080C6","#FFC20E","#FFFFFF"], 
    'LV' :["#000000","#A5ACAF"],
    'MIA':["#008E97","#FC4C02","#005778"], 
    'MIN':["#4F2683","#FFC62F"], 
    'NE' :["#002244","#C60C30","#B0B7BC"], 
    'NO' :["#101820","#D3BC8D"], 
    'NYG':["#0B2265","#A71930","#A5ACAF"], 
    'NYJ':["#125740","#000000","#FFFFFF"], 
    'PHI':["#004C54","#A5ACAF","#ACC0C6"], 
    'PIT':["#FFB612","#101820"], 
    'SEA':["#002244","#69BE28","#A5ACAF"], 
    'SF' :["#AA0000","#B3995D"],
    'TB' :["#D50A0A","#FF7900","#0A0A08"], 
    'TEN':["#0C2340","#4B92DB","#C8102E"], 
    'WAS':["#5A1414","#FFB612"], 
    'football':["#CBB67C","#663831"]
}

In [ ]:
def astar(grid, init, goal, cost):
    # Copy and paste your solution code from the previous exercise (#10)
    # TODO: CHANGE/UPDATE CODE
    
    import math
    delta = []
    #[[-1, 0],  # go up
    #     [0, -1],  # go left
    #     [1, 0],  # go down
    #     [0, 1],
    #     [1, 1],
    #     [1, -1],
    #     [-1, 1],
    #     [-1, -1]]  # go right'''
    for dir in range(0, 360, 90):
        delta.append([math.cos(dir), math.sin(dir)])
    
    path = None
    expand = []
    policy = np.ones_like(grid) * 9
    expand = np.ones_like(grid) * -1
    action = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    # TODO: ADD CODE HERE
    closed_list = [[0 for col in range(len(grid[0]))]for row in range(len(grid))]
    closed_list[init[0]][init[1]] = 1
    x = init[0]
    y = init[1]
    g = 0
    
    open_list = [[g, x, y]]
    
    found = False
    resign = False
    step = 0
    while found is False and resign is False:
        print(step)
        if len(open_list) == 0:
            resign = True
            path = 'fail'
            return None
        else:
            open_list.sort()
            open_list.reverse()
            next_node = open_list.pop()
            x = next_node[1]
            y = next_node[2]
            g = next_node[0]
            expand[x, y] = step
            if x == goal[0] and y == goal[1]:
                found = True
                path = next_node
            else:
                for i in range(len(delta)):
                    x2 = int(x + delta[i][0])
                    y2 = int(y + delta[i][1])
                    if 0<=x2 and x2 < len(grid) and 0<=y2 and y2<len(grid[0]):
                        if closed_list[x2][y2] ==0 and grid[x2][y2] == 0:
                            g2 = g + cost[x2][y2]
                            #else:
                            #    g2 = g + 2 * cost
                            open_list.append([g2, x2, y2])
                            closed_list[x2][y2] = 1
                            action[x2][y2] = i
                            
        step = step + 1
    fullpath = []
    x = int(path[1])
    y = int(path[2])
    fullpath.insert(0, [x, y])
    while x != init[0] or y != init[1]:
        x2 = x - delta[action[int(x)][int(y)]][0]
        y2 = y - delta[action[int(x)][int(y)]][1]
        x = x2
        y = y2
        fullpath.insert(0, [x, y])
        print(x, y)
    #    for y in range(expand.shape[0])
    return fullpath

def smooth(path, weight_data=0.5, weight_smooth=0.1, tolerance=0.000001):
    # Make a deep copy of path into newpath
    y = [np.array(p) for p in deepcopy(path)]
    # TODO: ADD CODE HERE
    newpath_i = [np.array(y[i]) - weight_smooth * (y[i+1] + y[i-1] - 2 * y[i]) for i in range(1,len(y)-1)]
    newpath_i.insert(0, y[0])
    newpath_i.append(y[-1])
    while np.linalg.norm(np.array(newpath_i) - np.array(y)) >= tolerance:
        y = newpath_i
        newpath_i = [weight_data * (path[i] - y[i]) + np.array(y[i]) + weight_smooth * (y[i+1] + y[i-1] - 2 * y[i]) for i in range(1,len(y)-1)]
        newpath_i.insert(0, y[0])
        newpath_i.append(y[-1])
    return y
    
def get_path_to_endzone(gameId, playId, frameId_):
    play = tracking_data[tracking_data['playId'] == playId][tracking_data['gameId'] == gameId]
    defense = plays[plays['playId'] == playId][plays['gameId'] == gameId]['defensiveTeam'].values[0]
    offense = plays[plays['playId'] == playId][plays['gameId'] == gameId]['possessionTeam'].values[0]
    defense = play[play['club'] == defense]
    offense = play[play['club'] == offense]
    if len(defense[defense['event'] == 'tackle']) > 0 and len(play[play['event'] == 'tackle'][play['club'] == 'football']) > 0:
        print(frameId_)
        frameId = defense[defense['event'] == 'tackle']['frameId'].values[0]
        defense = defense[defense['frameId'] == frameId_]
        offense = offense[offense['frameId'] == frameId_]
        runnerId = plays[plays['gameId'] == gameId][plays['playId'] == playId]['ballCarrierId'].values[0]
        football = play[play['frameId'] == frameId_][play['nflId'] == runnerId]
        football = np.array([football['x'].values[0], football['y'].values[0]])
        tacklerId = tackles[tackles['gameId'] == gameId][tackles['playId'] == playId]['nflId'].values[0]
        tackler = defense[defense['nflId'] == tacklerId]
        defense = defense[defense['nflId'] != runnerId]
        
        x_grid, y_grid = np.meshgrid(np.linspace(0, 120, 120), np.linspace(0, 53, 53), indexing = 'ij')
        #runner_loc = np.array([offense[offense['nflId'] == runnerId]['y'].values[0], offense[offense['nflId'] == runnerId]['x'].values[0]])
        offense = offense[offense['nflId'] != runnerId]
        grid = np.zeros((120, 53))
        for x_d, y_d, s, dir  in zip(offense['y'], offense['x'], offense['s'], offense['dir']):
            #grid[int(y_d),int(x_d)] = 1
            #for off_y in range(-1, 1):
            #    for off_x in range(-5, 5):
            #        grid[max(min(int(y_d + off_y), 120), 0), max(min(int(x_d + off_x), 53), 0)] = 1
            x_grid, y_grid = np.meshgrid(np.linspace(0, 120, 120), np.linspace(0, 53, 53), indexing = 'ij')
            player_dist = multivariate_normal(np.array([y_d, x_d]), np.eye(2) * .5).pdf(np.dstack((x_grid, y_grid)))
            #for x in range(player_dist.shape[0]):
            #    player_dist[x, :] = player_dist[x, :]+ x
            grid = grid + player_dist[:, ::-1]
        #print(football)
        #print(grid[int(football[0]), int(football[1])])
        #plt.imshow(grid>=0.1, cmap = 'gray')
        #plt.scatter(int(football[1]), int(football[0]))
        #plt.show()
        path = astar(grid>= 0.1, init = [int(tackler['x'].values[0]), int(tackler['y'].values[0])], goal = [int(football[0]), int(football[1])], cost = np.ones_like(grid))#
        if path is None:
            return [[int(tackler['x'].values[0]), int(tackler['y'].values[0])]]
        return smooth(path)
    
def animate_play(gameId,playId):
    selected_game_df = games[games.gameId==gameId].copy()
    selected_play_df = play_df[(play_df.playId==playId)&(play_df.gameId==gameId)].copy()
    
    tracking_players_df = pd.merge(tracking_df,players,how="left",on = "nflId")
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()

    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()

    #print(selected_tracking_df.club)
    #print(selected_tracking_df)
    # get good color combos
    team_combos = list(set(selected_tracking_df.club.unique())-set(["football"]))
    #print(team_combos)
    color_orders = ColorPairs(team_combos[0],team_combos[1])
    
    # get play General information 
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    ## Fixing first down marker issue from last year
    if selected_tracking_df.playDirection.values[0] == "right":
        first_down_marker = line_of_scrimmage + selected_play_df.yardsToGo.values[0]
    else:
        first_down_marker = line_of_scrimmage - selected_play_df.yardsToGo.values[0]
    down = selected_play_df.down.values[0]
    quarter = selected_play_df.quarter.values[0]
    gameClock = selected_play_df.gameClock.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # initialize plotly start and stop buttons for animation
    updatemenus_dict = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }


    frames = []
    for frameId in sorted_frame_list:
        data = []

        path = np.array(get_path_to_endzone(gameId, playId, frameId))
        data.append(go.Scatter(x=path[:, 0], y=path[:, 1], mode='lines', name='Path'))
        # Add Numbers to Field 
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[53.5-5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add line of scrimage 
        data.append(
            go.Scatter(
                x=[line_of_scrimmage,line_of_scrimmage], 
                y=[0,53.5],
                line_dash='dash',
                line_color='blue',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add First down line 
        data.append(
            go.Scatter(
                x=[first_down_marker,first_down_marker], 
                y=[0,53.5],
                line_dash='dash',
                line_color='yellow',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add Endzone Colors 
        endzoneColors = {0:color_orders[selected_game_df.homeTeamAbbr.values[0]][0],
                         110:color_orders[selected_game_df.visitorTeamAbbr.values[0]][0]}
        for x_min in [0,110]:
            data.append(
                go.Scatter(
                    x=[x_min,x_min,x_min+10,x_min+10,x_min],
                    y=[0,53.5,53.5,0,0],
                    fill="toself",
                    fillcolor=endzoneColors[x_min],
                    mode="lines",
                    line=dict(
                        color="white",
                        width=3
                        ),
                    opacity=1,
                    showlegend= False,
                    hoverinfo ="skip"
                )
            )
        # Plot Players
        for team in selected_tracking_df.club.unique():
            plot_df = selected_tracking_df[(selected_tracking_df.club==team)&(selected_tracking_df.frameId==frameId)].copy()
            if team != "football":
                hover_text_array=[]
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId==nflId]
                    hover_text_array.append("nflId:{}<br>displayName:{}<br>Player Speed:{} yd/s".format(selected_player_df["nflId"].values[0],
                                                                                      players[players['nflId'] == selected_player_df['nflId'].to_numpy()[0]]['displayName'].values[0],
                                                                                      selected_player_df["s"].values[0]))
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker=go.scatter.Marker(
                                                                                             color=color_orders[team][0],
                                                                                             line=go.scatter.marker.Line(width=2,
                                                                                                            color=color_orders[team][1]),
                                                                                             size=10),
                                        name=team,hovertext=hover_text_array,hoverinfo="text"))
            else:
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker=go.scatter.Marker(
                                                                                             color=color_orders[team][0],
                                                                                             line=go.scatter.marker.Line(width=2,
                                                                                                            color=color_orders[team][1]),
                                                                                             size=10),
                                        name=team,hoverinfo='none'))
        # add frame to slider
        slider_step = {"args": [
            [frameId],
            {"frame": {"duration": 100, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": str(frameId),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]["data"],
        layout= layout,
        frames=frames[1:]
    )
    # Create First Down Markers 
    for y_val in [0,53]:
        fig.add_annotation(
                x=first_down_marker,
                y=y_val,
                text=str(down),
                showarrow=False,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="black"
                    ),
                align="center",
                bordercolor="black",
                borderwidth=2,
                borderpad=4,
                bgcolor="#ff7f0e",
                opacity=1
                )
    # Add Team Abbreviations in EndZone's
    for x_min in [0,110]:
        if x_min == 0:
            angle = 270
            teamName=selected_game_df.homeTeamAbbr.values[0]
        else:
            angle = 90
            teamName=selected_game_df.visitorTeamAbbr.values[0]
        fig.add_annotation(
            x=x_min+5,
            y=53.5/2,
            text=teamName,
            showarrow=False,
            font=dict(
                family="Courier New, monospace",
                size=32,
                color="White"
                ),
            textangle = angle
        )
    return fig